In [19]:
#importing the modules
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import ops
from shapely.geometry import Point, LineString, Polygon

In [10]:
#importing the data
data = pd.read_csv('/content/drive/MyDrive/occ.csv',
            delimiter = ';')

#filtering for sp and removing weird column
data = data[data['Taxon_type'] == 'sp']
data.drop('Unnamed: 0', axis = 1,inplace = True)

In [20]:
#transforming the points in coordinates
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.Longitude, data.Latitude))

In [58]:
#latin america polygon
world_gdf = gpd.read_file(
    gpd.datasets.get_path('naturalearth_lowres')
)

latam_continents = ['South America', 'Central America',
                    'Caribbean']

sudamerica_gdf = world_gdf[world_gdf.continent == 'South America']

In [84]:
#merging all the polygons
sudamerica_polygon = ops.unary_union(sudamerica_gdf.geometry)

In [89]:
len(gdf.geometry)

135671

In [93]:
#testing for an example point
range(0,len(gdf.geometry))


range(0, 135671)

In [95]:
contained_sudamerica = [sudamerica_polygon.contains(gdf.geometry[i]) for i in range(0,len(gdf.geometry))]

In [98]:
gdf['contained_sudamerica'] = contained_sudamerica

In [138]:
outside_sudamerica = gdf[gdf['contained_sudamerica'] == False]
inside_sudamerica = gdf[gdf['contained_sudamerica'] == True]

In [ ]:
#now i will invert the outside_sudamerica points and try to recover a few
outside_sudamerica.drop('geometry',axis = 1,inplace = True )


In [ ]:
outside_sudamerica['longitude'] = outside_sudamerica['Latitude']
outside_sudamerica['latitude'] = outside_sudamerica['Longitude']
outside_sudamerica.drop(['Latitude','Longitude'], axis=1, inplace = True)

In [112]:
#new coordinates
out_s_new_coords = gpd.GeoDataFrame(outside_sudamerica, geometry=gpd.points_from_xy(outside_sudamerica.longitude, outside_sudamerica.latitude))

In [123]:
out_s_new_coords.index = range(0,88124)

In [124]:
#checking if new points are inside 
rechecking_cont_sud = [sudamerica_polygon.contains(out_s_new_coords.geometry[i]) for i in range(0,len(out_s_new_coords.geometry))]

In [125]:
out_s_new_coords['repescagem'] = rechecking_cont_sud

In [136]:
final_repescagem = out_s_new_coords[out_s_new_coords['repescagem'] == True]

#70469 of the 88 thousand were in south america

In [139]:
#preparing the two dataset to be merged together: the first one is the one which originally had
#all the points already in there
inside_sudamerica.drop(['geometry','contained_sudamerica'],axis = 1, inplace = True)
#the second one is the one which when inverted, the points showed up in south america
final_repescagem['Latitude'] = final_repescagem['latitude']
final_repescagem['Longitude'] = final_repescagem['longitude']

final_repescagem.drop(['repescagem','contained_sudamerica', 'latitude', 'longitude', 'geometry'],axis = 1, inplace = True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [141]:
data_corrected_final = inside_sudamerica.append(final_repescagem)

In [142]:
data_corrected_final.to_csv("/content/drive/MyDrive/data_corrected.csv")